# Framework Backtest: Fear & Greed Strategy

This notebook conducts a backtest of the `FearGreedStrategy` using the components developed in the framework (`MarketDataClient`, `SentimentAnalyzer`). The goal is to validate the performance of our specific Fear & Greed formula against historical data.

## 1. Setup & Imports

We need to adjust the system path to import our framework modules from the `src` directory.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_ingestion.market_data_client import MarketDataClient
from src.sentiment_engine.sentiment_analyzer import SentimentAnalyzer
from src.strategies.fear_greed_strategy import FearGreedStrategy
from alpaca_trade_api.rest import TimeFrame

print("Framework modules imported successfully.")

## 2. Data Fetching & Index Calculation

We will fetch a long history of SPY data and then run our `SentimentAnalyzer` over it to generate a time series of the Fear & Greed Index.

In [ ]:
# Initialize clients
market_client = MarketDataClient()
sentiment_analyzer = SentimentAnalyzer(market_client)

# Fetch historical data for SPY
start_date = '2007-01-01'
end_date = '2024-01-01'
spy_bars = market_client.get_historical_bars('SPY', TimeFrame.Day, start_date, end_date)

if spy_bars is not None:
    print(f"Successfully fetched {len(spy_bars)} bars of SPY data.")
    # Calculate F&G Index for each day in our dataset
    # This is a simplified approach; a real implementation would be more efficient
    fng_values = []
    for i in range(200, len(spy_bars)):
        # Pass a rolling window of data to the analyzer
        # Note: This is a mock of how the analyzer would work on historical data
        # We are recalculating scores based on expanding history
        temp_analyzer = SentimentAnalyzer(market_client) # Re-init to avoid state issues
        temp_analyzer.market_data_client.get_historical_bars = lambda s, tf, st, ed: spy_bars.iloc[:i]
        fng = temp_analyzer.calculate_fear_greed_index('SPY')
        fng_values.append(fng)
    
    # Align the F&G values with the dataframe
    spy_bars = spy_bars.iloc[200:].copy()
    spy_bars['fng_index'] = fng_values
    print("Fear & Greed Index calculated for the historical period.")
else:
    print("Failed to fetch data.")

## 3. Strategy & Signal Generation

Now, we apply our `FearGreedStrategy` to the calculated index values.

In [ ]:
strategy = FearGreedStrategy(buy_threshold=35, sell_threshold=65)

# Generate signals
spy_bars['signal'] = spy_bars['fng_index'].apply(strategy.generate_signal)

# Convert signals to positions (-1 for SELL, 1 for BUY, 0 for HOLD)
spy_bars['position'] = 0
spy_bars.loc[spy_bars['signal'] == 'BUY', 'position'] = 1
spy_bars.loc[spy_bars['signal'] == 'SELL', 'position'] = -1

# Shift positions to avoid lookahead bias (trade on next day's open)
spy_bars['position'] = spy_bars['position'].shift(1)
spy_bars.dropna(inplace=True)

print("Signals and positions generated:")
print(spy_bars['signal'].value_counts())

## 4. Backtest Execution & Performance Metrics

In [ ]:
# Calculate returns
spy_bars['spy_returns'] = spy_bars['close'].pct_change()
spy_bars['strategy_returns'] = spy_bars['spy_returns'] * spy_bars['position']

# Calculate cumulative returns
spy_bars['buy_and_hold'] = (1 + spy_bars['spy_returns']).cumprod()
spy_bars['strategy_cumulative'] = (1 + spy_bars['strategy_returns']).cumprod()

# --- Performance Metrics ---
final_bnh_return = spy_bars['buy_and_hold'].iloc[-1]
final_strategy_return = spy_bars['strategy_cumulative'].iloc[-1]

# Sharpe Ratio (assuming risk-free rate is 0)
sharpe_ratio = (spy_bars['strategy_returns'].mean() / spy_bars['strategy_returns'].std()) * np.sqrt(252)

# Max Drawdown
rolling_max = spy_bars['strategy_cumulative'].cummax()
daily_drawdown = spy_bars['strategy_cumulative'] / rolling_max - 1.0
max_drawdown = daily_drawdown.min()

print("--- Backtest Results ---")
print(f"Buy and Hold Final Return: {final_bnh_return:.2f}")
print(f"Strategy Final Return: {final_strategy_return:.2f}")
print(f"Strategy Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Strategy Max Drawdown: {max_drawdown:.2%}")

## 5. Performance Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [3, 1]})

# Plot cumulative returns
ax1.plot(spy_bars['buy_and_hold'], label='Buy and Hold (SPY)')
ax1.plot(spy_bars['strategy_cumulative'], label='Fear & Greed Strategy')
ax1.set_title('Strategy Performance vs. Buy and Hold')
ax1.set_ylabel('Cumulative Returns')
ax1.legend()
ax1.grid(True)

# Plot the Fear & Greed Index
ax2.plot(spy_bars['fng_index'], label='Fear & Greed Index', color='orange')
ax2.axhline(strategy.buy_threshold, color='green', linestyle='--', label=f'Buy Threshold ({strategy.buy_threshold})')
ax2.axhline(strategy.sell_threshold, color='red', linestyle='--', label=f'Sell Threshold ({strategy.sell_threshold})')
ax2.set_title('Fear & Greed Index Over Time')
ax2.set_xlabel('Date')
ax2.set_ylabel('Index Value')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()